<a href="https://colab.research.google.com/github/andriellymoraespereira/analise-de-visitas-em-ecommerce/blob/main/01_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"andrielly","key":"7dd932a3b2f2dbb561692b726ebb7f5d"}'}

In [2]:
import os

# Criar pasta .kaggle
!mkdir -p ~/.kaggle

# Mover o kaggle.json para a pasta .kaggle
!mv kaggle.json ~/.kaggle/

# Definir permissões adequadas para o arquivo
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
# Baixar o dataset específico do Kaggle
!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
... resuming from 1689255936 bytes (2917464971 bytes left) ...
100% 4.29G/4.29G [00:27<00:00, 91.8MB/s]
100% 4.29G/4.29G [00:27<00:00, 105MB/s] 


In [4]:
# Descompactar o arquivo
import zipfile

# Descompactar o arquivo ZIP
with zipfile.ZipFile('ecommerce-behavior-data-from-multi-category-store.zip', 'r') as zip_ref:
    zip_ref.extractall('ecommerce_data')

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Carregar um subset dos dados
file_path = 'ecommerce_data/2019-Nov.csv'
chunk_size = 1000000
chunks = []

for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    chunks.append(chunk)
    if len(chunks) >= 5:  # Carregar mais registros (5 chunks)
        break

df = pd.concat(chunks, axis=0)

In [6]:
# Verificar valores ausentes
print("Valores ausentes antes do preenchimento:\n", df.isnull().sum())

Valores ausentes antes do preenchimento:
 event_time             0
event_type             0
product_id             0
category_id            0
category_code    1602394
brand             727867
price                  0
user_id                0
user_session           0
dtype: int64


In [7]:
# Preencher valores ausentes nas colunas category_code e brand
df['category_code'].fillna('Unknown', inplace=True)
df['brand'].fillna('Unknown', inplace=True)

In [8]:
# Verificar e remover duplicatas baseadas em colunas que indicam o mesmo evento
print(f"Número de linhas antes da remoção de duplicatas: {df.shape[0]}")
df_cleaned = df.drop_duplicates(subset=['user_id', 'event_time', 'category_code', 'brand', 'price'])
print(f"Número de linhas após a remoção de duplicatas: {df_cleaned.shape[0]}")

Número de linhas antes da remoção de duplicatas: 5000000
Número de linhas após a remoção de duplicatas: 4997407


In [9]:
# Verificar se ainda há valores ausentes
print("Valores ausentes após o preenchimento e remoção de duplicatas:\n", df_cleaned.isnull().sum())

Valores ausentes após o preenchimento e remoção de duplicatas:
 event_time       0
event_type       0
product_id       0
category_id      0
category_code    0
brand            0
price            0
user_id          0
user_session     0
dtype: int64


In [10]:
# Corrigir tipos de dados, se necessário (exemplo: converter para datetime)
df_cleaned['event_time'] = pd.to_datetime(df_cleaned['event_time'])
if 'start_time' in df_cleaned.columns:
    df_cleaned['start_time'] = pd.to_datetime(df_cleaned['start_time'])
if 'end_time' in df_cleaned.columns:
    df_cleaned['end_time'] = pd.to_datetime(df_cleaned['end_time'])

<ipython-input-10-e4cbd4256b4c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['event_time'] = pd.to_datetime(df_cleaned['event_time'])


In [11]:
# Exemplo: verificar valores negativos em colunas que não deveriam tê-los
df_cleaned = df_cleaned[df_cleaned['price'] >= 0]

In [12]:
# Salvar o conjunto de dados limpo
df_cleaned.to_csv('cleaned_data.csv', index=False)

print("Dados limpos e salvos com sucesso.")

Dados limpos e salvos com sucesso.
